In [1]:
from web_search import (
    get_page_content,
    process_search_results,
    generate_queries_search_engine,
)
from dotenv import load_dotenv

load_dotenv()

c:\Users\phi.nguyen\.conda\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [12]:
# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal

from langchain_core.tools import tool


@tool
def get_page_content_tool(input: str) -> str:
    """
       fetch the html content from provided url, must be used after process_search_results_tool
    Args:
       input (str): An url to parse.
    """
    return get_page_content(input)  # Ensure this returns the content


@tool
def process_search_results_tool(input: list) -> list:
    """
    search the input in the internet

    This function decomposes the input query into sub-questions, retrieves search results for each sub-question,
    and compiles a list of URLs along with their frequency of occurrence, age, and description.

    Args:
        input (str): The input query string to be processed.

    Returns:
        list: A sorted list of dictionaries containing URLs, each with its frequency of occurrence.
    """

    return process_search_results(input)


tools = [get_page_content_tool, process_search_results_tool]


In [13]:
from system_prompt import DEFAULT_SYSTEM_PROMPT
import datetime

DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)
prompt = DEFAULT_SYSTEM_PROMPT_WITH_TIME


In [14]:
# Define the graph
from langgraph.prebuilt import create_react_agent

# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)
model.bind_tools(tools)


RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C233179820>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C23318E6D0>, root_client=<openai.OpenAI object at 0x000001C23316F910>, root_async_client=<openai.AsyncOpenAI object at 0x000001C233179F10>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_page_content_tool', 'description': 'fetch the html content from provided url, must be used after process_search_results_tool\nArgs:\n   input (str): An url to parse.', 'parameters': {'properties': {'input': {'type': 'string'}}, 'required': ['input'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'process_search_results_tool', 'description': 'search the input in the internet\n\nThis function decomposes the input query into sub-questions, retrieves search results for each sub-qu

In [15]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    # other params...
)
model = model.bind_tools(tools)

In [16]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    """The state of the agent."""

    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]


In [17]:
import json
from langchain_core.messages import ToolMessage, SystemMessage
from langchain_core.runnables import RunnableConfig

tools_by_name = {tool.name: tool for tool in tools}


# Define our tool node
def tool_node(state: AgentState):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}


# Define the node that calls the model
def call_model(
    state: AgentState,
    config: RunnableConfig,
):
    # this is similar to customizing the create_react_agent with state_modifier, but is a lot more flexible
    system_prompt = SystemMessage(
        "You are a helpful AI assistant, please respond to the users query to the best of your ability!"
    )
    response = model.invoke([system_prompt] + state["messages"], config)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the conditional edge that determines whether to continue or not
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


In [18]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", "agent")

# Now we can compile and visualize our graph
graph = workflow.compile()


In [21]:
# Helper function for formatting the stream nicely
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", "thế còn chủ tịch nước tìm site khác đi
                        ?")]}
print_stream(graph.stream(inputs, stream_mode="values"))


================================ Human Message =================================

thế còn chủ tịch nước?
================================== Ai Message ==================================

[{'text': 'Để trả lời câu hỏi của bạn về chủ tịch nước Việt Nam, tôi cần tìm kiếm thông tin cập nhật nhất. Tôi sẽ sử dụng công cụ tìm kiếm để lấy thông tin mới nhất về chủ tịch nước Việt Nam hiện tại.', 'type': 'text'}, {'id': 'toolu_01UxpKfCQHGgziFQ5KRuE1GN', 'input': {'input': ['Chủ tịch nước Việt Nam hiện tại là ai?', 'Thông tin về chủ tịch nước Việt Nam']}, 'name': 'process_search_results_tool', 'type': 'tool_use'}]
Tool Calls:
  process_search_results_tool (toolu_01UxpKfCQHGgziFQ5KRuE1GN)
 Call ID: toolu_01UxpKfCQHGgziFQ5KRuE1GN
  Args:
    input: ['Chủ tịch nước Việt Nam hiện tại là ai?', 'Thông tin về chủ tịch nước Việt Nam']
sub question: [['Chủ tịch nước Việt Nam hiện tại là ai?', 'Thông tin về chủ tịch nước Việt Nam']]
================================= Tool Message ===========================

SSLError: HTTPSConnectionPool(host='vpctn.gov.vn', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1147)')))